# Tugas Multiprocessing Komputasi Paralel kelas D
##### Peter Abednego Wijaya 			235150300111013
##### Gilang Shido Faizalhaq			235150300111011
##### Rizqy Jauhary Atsaany			    235150300111038

# 1: Impor Library

In [1]:
import time                 # Untuk mengukur waktu eksekusi
import string               # Untuk membantu membersihkan tanda baca
import re                   # Regular Expression, untuk menemukan kata
from collections import Counter  # Struktur data yang sangat efisien untuk menghitung
import multiprocessing      # Library utama untuk pemrosesan paralel
import os                   # Untuk berinteraksi dengan sistem operasi (misal: cek file, cek core CPU)

#  2. Cek File data.txt

In [2]:
file_name = r"P:\Tugas\Tugas Kuliah\Kompar\data.txt"

print(f"Mencari file '{file_name}'...")

if os.path.exists(file_name):
    print(f" File '{file_name}' ditemukan.")
else:
    print(f"File '{file_name}' tidak ditemukan.")

Mencari file 'P:\Tugas\Tugas Kuliah\Kompar\data.txt'...
 File 'P:\Tugas\Tugas Kuliah\Kompar\data.txt' ditemukan.


# 3. Fungsi Bantuan untuk Memproses Teks

In [3]:
# Menggunakan regex \b[a-zA-Z]+\b untuk mengekstrak "kata"
# yang hanya terdiri dari huruf. Ini akan mengabaikan angka dan tanda baca.
# \b = batas kata (word boundary)
# [a-zA-Z]+ = satu atau lebih huruf (case-insensitive)
WORD_REGEX = re.compile(r'\b[a-zA-Z]+\b')

def process_text_chunk(lines):
    """
    Mengambil daftar baris teks, menggabungkannya, membersihkan,
    dan menghitung frekuensi kata.
    Mengembalikan objek collections.Counter.
    """
    # 1. Gabungkan semua baris menjadi satu string besar dan ubah ke huruf kecil
    text = ' '.join(lines).lower()

    # 2. Temukan semua kata (hanya huruf) menggunakan regex
    words = WORD_REGEX.findall(text)

    # 3. Hitung frekuensi kata dan kembalikan
    return Counter(words)

In [ ]:
# Baca File Hanya Sekali
print(f"Membaca file '{file_name}' ke dalam memori...")
start_read = time.time()

with open(file_name, 'r', encoding='utf-8') as f:
    all_lines = f.readlines()

end_read = time.time()
print(f"Selesai membaca {len(all_lines)} baris dalam {end_read - start_read:.4f} detik.")

Membaca file 'P:\Tugas\Tugas Kuliah\Kompar\data.txt' ke dalam memori...
Selesai membaca 7211778 baris dalam 2.4763 detik.


# 4. Metode serial

In [5]:
# Sel 5: Metode Serial (Satu Proses)

print("Memulai penghitungan SERIAL...")
# Catat waktu mulai
start_serial = time.time()

# 1. Proses seluruh variabel 'all_lines' menggunakan satu core.
#    (Kita TIDAK membaca file lagi di sini)
serial_counts = process_text_chunk(all_lines)

# 2. Ambil 20 kata teratas dari hasil Counter
top_20_serial = serial_counts.most_common(20)

# 3. Catat waktu selesai
end_serial = time.time()
# Hitung total waktu eksekusi
serial_time = end_serial - start_serial

print(f"Penghitungan SERIAL selesai dalam {serial_time:.4f} detik.\n")
print("--- 20 Kata Teratas (Serial) ---")
for word, count in top_20_serial:
    print(f"{word}: {count}")

Memulai penghitungan SERIAL...
Penghitungan SERIAL selesai dalam 39.1367 detik.

--- 20 Kata Teratas (Serial) ---
the: 2378901
and: 1527477
of: 1480983
to: 1077741
a: 903504
in: 788802
i: 782418
that: 555387
it: 439782
you: 425607
is: 402003
for: 373149
with: 366870
his: 365043
as: 364245
he: 351918
was: 338940
not: 331842
my: 331338
s: 321048


# 5. Metode paralel

In [ ]:
# Sel 6: Metode Paralel (Multiprocessing)

print("\nMemulai penghitungan PARALEL...")
# Catat waktu mulai
start_parallel = time.time()

# Tentukan jumlah proses yang akan digunakan
# Di Colab, ini biasanya 2 atau 4
num_processes = multiprocessing.cpu_count()
print(f"Menggunakan {num_processes} core CPU...")

# 1. Bagi data (variabel 'all_lines' yang sama) menjadi 'num_processes' chunks
# (len(all_lines) + num_processes - 1) // num_processes adalah trik
# untuk membagi secara merata dan menangani sisa bagi
chunk_size = (len(all_lines) + num_processes - 1) // num_processes
# Gunakan list comprehension untuk memotong 'all_lines'
chunks = [all_lines[i:i + chunk_size] for i in range(0, len(all_lines), chunk_size)]
print(f"Data dibagi menjadi {len(chunks)} chunks, masing-masing ~{chunk_size} baris.")

# 2. Buat Pool proses (kolam pekerja)
# 'with' statement akan otomatis mengelola (memulai & menutup) pool
with multiprocessing.Pool(processes=num_processes) as pool:

    # 3. Tahap MAP:
    # pool.map() akan "memetakan" fungsi 'process_text_chunk'
    # ke setiap item dalam 'chunks'.
    # Jika ada 4 core & 4 chunk, Core 1 mengerjakan Chunk 1, Core 2 mengerjakan Chunk 2, dst.
    # Ini adalah langkah yang memblokir (menunggu) sampai SEMUA proses selesai.
    # 'results' adalah list berisi hasil Counter dari tiap proses.
    # [Counter_dari_Core1, Counter_dari_Core2, ...]
    results = pool.map(process_text_chunk, chunks)

# 4. Tahap REDUCE:
# Sekarang kita harus menggabungkan semua hasil Counter dari 'results'
print("Menggabungkan hasil dari semua proses...")
parallel_counts = Counter() # Buat satu Counter kosong
for count_chunk in results: # Loop untuk setiap Counter di dalam 'results'
    parallel_counts.update(count_chunk) # Gabungkan isinya ke Counter utama

# 5. Dapatkan 20 kata teratas
top_20_parallel = parallel_counts.most_common(20)

# 6. Catat waktu selesai
end_parallel = time.time()
parallel_time = end_parallel - start_parallel

print(f"Penghitungan PARALEL selesai dalam {parallel_time:.4f} detik.\n")
print("--- 20 Kata Teratas (Paralel) ---")
for word, count in top_20_parallel:
    print(f"{word}: {count}")


Memulai penghitungan PARALEL...
Menggunakan 4 core CPU...
Data dibagi menjadi 4 chunks, masing-masing ~1802945 baris.


# 6. Perbandingan waktu eksekusi

In [7]:
# Sel 7: Perbandingan

print("--- Perbandingan Waktu Eksekusi ---")
print(f"Waktu Serial:   {serial_time:.4f} detik")
print(f"Waktu Paralel:  {parallel_time:.4f} detik")

# Hitung speedup (Waktu Serial dibagi Waktu Paralel)
if parallel_time > 0:
    speedup = serial_time / parallel_time
    print(f"\nSpeedup: {speedup:.2f}x")
    if speedup > 1:
        print(f"Metode paralel {speedup:.2f} kali lebih cepat daripada metode serial.")
    else:
        # Ini terjadi jika file terlalu kecil, overhead paralel lebih besar
        print("Metode paralel lebih lambat daripada serial.")
else:
    print("\nTidak dapat menghitung speedup (waktu paralel adalah nol).")

# Verifikasi hasil (sekarang seharusnya identik)
if top_20_serial == top_20_parallel:
    print("Verifikasi BERHASIL: Hasil serial dan paralel identik.")
else:
    print("Verifikasi GAGAL: Hasil serial dan paralel berbeda.")

--- Perbandingan Waktu Eksekusi ---
Waktu Serial:   38.0027 detik
Waktu Paralel:  39.4921 detik

Speedup: 0.96x
Metode paralel lebih lambat daripada serial.
Verifikasi BERHASIL: Hasil serial dan paralel identik.
